In [11]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [10]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../../data/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)


## Do your thing crazy machine learning thing here :) ...

In [12]:
def build_poly(x, degree):
    
    matrix = np.zeros((x.shape[0], x.shape[1] * (degree + 1)))
    
    for i in range(degree + 1):
        matrix[:, (i * x.shape[1]) : ((i + 1) * x.shape[1])] = (x ** i)[:]
    return matrix


def build_k_indices(y, k_fold, seed=62):
    """build k indices for k-fold."""
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval]
                 for k in range(k_fold)]
    return np.array(k_indices)

def batch_iter(y, tx, batch_size, num_batches=None, shuffle=True):
    """
    Generate a minibatch iterator for a dataset.
    Takes as input two iterables (here the output desired values 'y' and the input data 'tx')
    Outputs an iterator which gives mini-batches of `batch_size` matching elements from `y` and `tx`.
    Data can be randomly shuffled to avoid ordering in the original data messing with the randomness of the minibatches.
    Example of use :
    for minibatch_y, minibatch_tx in batch_iter(y, tx, 32):
        <DO-SOMETHING>
    """

    data_size = len(y)
    num_batches_max = int(np.ceil(data_size/batch_size))
    if num_batches is None:
        num_batches = num_batches_max
    else:
        num_batches = min(num_batches, num_batches_max)

    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_y = y[shuffle_indices]
        shuffled_tx = tx[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_tx = tx
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]

In [75]:
BINARY_CLASSIFICATOIN_0 = -1
BINARY_CLASSIFICATOIN_1 = 1


def sigmoid(t):
    """apply sigmoid function on t."""
    return 1.0 / (1.0 + np.exp(-t))


def calculate_loss_logistic_regression(y, tx, w):
    """compute the cost by negative log likelihood."""
    prediction = tx @ w
    
    y1 = np.where(y == BINARY_CLASSIFICATOIN_1)

    over_700 = np.where(prediction >= 700)

    prediction_result = np.log(1 + np.exp(prediction))
    prediction_result[over_700] = prediction[over_700]
    prediction_result[y1] -= prediction[y1]
    
    result = np.sum(prediction_result)
    return result


def calculate_gradient_logistic_regression(y, tx, w):
    """compute the gradient of loss."""

    y1 = np.where(y == BINARY_CLASSIFICATOIN_1)
    sig = sigmoid(tx @ w).reshape(len(y))
    sig[y1] -= y[y1]

    return (tx.T @ sig).reshape((tx.shape[1], 1))


def calculate_hessian_logistic_regression(y, tx, w):
    """return the hessian of the loss function."""
    Snn = (sigmoid(tx @ w) * (1 - sigmoid(tx @ w)))
    
    
    Snn = Snn.reshape(Snn.shape[0])
    S = np.diag(Snn)
    return (tx.T @ S @ tx).reshape((tx.shape[1], tx.shape[1]))
    
    
def line_search_gamma(loss, loss_prev, gamma):
    if (loss > loss_prev):
        gamma = gamma / 1.5
    return gamma
    

def logistic_regression_helper(y, tx, gamma, max_iters, lambda_):

    w = np.zeros((tx.shape[1], 1))
    threshold = 1e-8
    loss_prev = 0
    w_max = w
    perf = 0
    i = 0
    
    batch_size = len(y) / 5
    counter = 0

    for iter in range(max_iters):
                
#         for mini_y, mini_x in batch_iter(y, tx, batch_size):
        loss = calculate_loss_logistic_regression(y, tx, w) + lambda_ * np.linalg.norm(w, 2)
        gradient = calculate_gradient_logistic_regression(y, tx, w)
#         loss = calculate_loss_logistic_regression(mini_y, mini_x, w) + lambda_ * np.linalg.norm(w, 2)
#         gradient = calculate_gradient_logistic_regression(mini_y, mini_x, w)
#             hessian = calculate_hessian_logistic_regression(mini_y, mini_x, w)

        w -= gradient * gamma

        if (loss_prev != 0) and np.abs(loss_prev - loss) < threshold:
            print("Reached Theshold, exit")
            break
        
        gamma = line_search_gamma(loss, loss_prev, gamma)
            
        loss_prev = loss
        
#         if (iter > 0 and iter < 10000 and (iter % 3000) == 0):
#             gamma = gamma / 10
#             print("Gamma / 10")
        
#         if ((iter >= 10000 and iter <= 24000) and (iter % 1000) == 0):
#             counter -= 1
#             if (np.abs(loss_prev - loss) > 5000):
#                 gamma = gamma / 10
#                 print("Gamma / 10")

#             if (np.abs(loss_prev - loss) < 1000 and counter == 0):
#                 counter = 3
#                 gamma = gamma * 2
#                 print("Gamma * 2")

        
#         if ((iter >= 25000) and (iter % 100) == 0):
#             counter -= 1
#             if (np.abs(loss_prev - loss) > 10000):
#                 gamma = gamma / 10
#                 print("Gamma / 10")

#             if (np.abs(loss_prev - loss) < 1000 and counter == 0):
#                 counter = 3
#                 gamma = gamma * 2
#                 print("Gamma * 2")
            
        
#         if (iter % 10) == 0:
#             cur_perf = performance(w, y, tx)
#             if cur_perf >= perf:
#                 w_max = w
#                 perf = cur_perf
#                 i = iter


        if (iter % 100) == 0:
            print("Gamma: ", gamma)
            print("Current iteration={i}, the loss={l}".format(i=iter, l=loss))

#         if (iter % 300) == 0:
#             print(w_max)
#             print("Performance: ", perf)
#             print("Iteration: ", i)
            

    return w


def logistic_regression(y, tx, gamma, max_iters):
    """ return the final w from the logistic regression """
    return logistic_regression_helper(y, tx, gamma, max_iters, lambda_=0)


def reg_logistic_regression(y, tx, lambda_, gamma, max_iters):
    """ return the final w from the penalized logistic regression, with lambda_ as a non 0 value"""
    return logistic_regression_helper(y, tx, gamma, max_iters, lambda_)


In [76]:
def performance(weights, y, xT):
    """Returns the percentage of successful classifications for the weights,
    given the expected results (y) and data (xT)"""
    from proj1_helpers import predict_labels
    compare_pred = predict_labels(weights, xT)
    compare_pred -= y.reshape((len(y), 1))
        
    non_zero = 0
    for i in range(len(compare_pred)):
        if compare_pred[i] != 0:
            non_zero += 1
            
    return 1 - non_zero / compare_pred.size

In [77]:

def plot_feature_relationship(y, tX):

    std_tx = standardize(tX)

    row = np.zeros(std_tx.shape[0])

    # for i in range(len(tX)):
    #     row[i] = tX[i][0]

    plt.figure(figsize=(20, 20))

    for j in range(std_tx.shape[1]):
        for i in range(len(std_tx)):
            row[i] = std_tx[i][j]
        plt.subplot(5, 6, j + 1)
        plt.title(j)
        plt.plot(row[np.where(y == 1)], y[np.where(y == 1)], 'ro')
        plt.plot(row[np.where(y == -1)], y[np.where(y == -1)], 'bo')
  

    plt.tight_layout()

    plt.show()

# plot_feature_relationship(y, tX)


In [78]:

# def standardize(x):
# #     cols = [4, 5, 6, 12, 26, 28]
# #     for i in range(len(x)):
# #         x[i][np.where(x[i] == -999)] = 0

#     # Combine feature, or do poly expansion
    
#     # Do not normalize the jet num
#     jet_num = x[:, jet_num_col]
    
#     # Replace -999 with some value that is the mean/median of the represent dataset 
#     for i in range(x.shape[1]):
#         median = np.median(x[np.where(x[:, i] != -999), i])
#         x[np.where(x[:, i] == -999), i] = median 
#         x[np.where(x[:, i] != -999), i] = x[np.where(x[:, i] != -999), i] - np.mean(x[np.where(x[:, i] != -999), i])
    
#     mean_x = np.mean(x, axis=0)
#     x = x - mean_x
    
#     std_x = np.std(x, axis=0)
#     x[:, std_x > 0] = x[:, std_x > 0] / std_x[std_x > 0]

#     x[:, jet_num_col] = jet_num
    
#     return x

# print(standardize(tX) - tX)

In [79]:
# def standardize_01(x):
# #     cols = [4, 5, 6, 12, 26, 28]
# #     for i in range(len(x)):
# #         x[i][np.where(x[i] == -999)] = 0

#     # Combine feature, or do poly expansion
    
#     # Do not normalize the jet num
    
#     # Replace -999 with some value that is the mean/median of the represent dataset 
#     for i in range(x.shape[1]):
#         x[np.where(x[:, i] == -999), i] = 1
    
#     mean_x = np.mean(x, axis=0)
#     x = x - mean_x
    
#     std_x = np.std(x, axis=0)
#     x[:, std_x > 0] = x[:, std_x > 0] / std_x[std_x > 0]

#     x[:, jet_num_col] = 1
    
#     return x


In [80]:
def standardize_0123_helper(x):
    for i in range(x.shape[1]):
        mean = np.mean(x[np.where(x[:, i] != -999), i])
        x[np.where(x[:, i] == -999), i] = mean 
        x[np.where(x[:, i] != -999), i] = x[np.where(x[:, i] != -999), i] - mean
    
    std_x = np.std(x, axis=0)
    x[:, std_x > 0] = x[:, std_x > 0] / std_x[std_x > 0]
    
    return x


def standardize_0(x):
    feature_left = np.array([0, 1, 2, 3, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21])
    left_x = np.zeros((x.shape[0], len(feature_left)))
    left_x[:, :] = x[:, feature_left]
    return standardize_0123_helper(left_x)
    
    

def standardize_1(x):
    feature_left = np.array([0, 1, 2, 3, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 29])
    left_x = np.zeros((x.shape[0], len(feature_left)))
    left_x[:, :] = x[:, feature_left]
    return standardize_0123_helper(left_x)
    
    
def standardize_23(x):
    feature_left = np.delete(np.arange(30), 22)
    left_x = np.zeros((x.shape[0], len(feature_left)))
    left_x[:, :] = x[:, feature_left]
    return standardize_0123_helper(left_x)

    

In [81]:
jet_num_col = 22


def split_dataset_wrt22(x):
    x_22_0 = np.where(x[:, jet_num_col] == 0)
    x_22_1 = np.where(x[:, jet_num_col] == 1)
    x_22_23 = np.where(x[:, jet_num_col] >= 2)
    return x_22_0, x_22_1, x_22_23



In [82]:


# There are two parameters, lambda_ and gamma, where gamma is the step size 

# max_iter = 15000
# lambdas = np.arange(0.1, 0.4, 0.1)
# gammas = np.array([0.004])

# degree = 3

# i_0, i_1, i_23 = split_dataset_wrt22(tX)

# tx_0 =  tX[i_0]
# y_0 =   y[i_0]
# tx_1 =  tX[i_1]
# y_1 =   y[i_1]
# tx_23 = tX[i_23]
# y_23 =  y[i_23]

# std_tx_0 = standardize_01(tx_0)
# std_tx_1 = standardize_01(tx_1)
# std_tx_23 = standardize(tx_23)

# matrix_std_tx_0 = build_poly(std_tx_0, degree)
# matrix_std_tx_1 = build_poly(std_tx_1, degree)
# matrix_std_tx_23 = build_poly(std_tx_23, degree)


# print(std_tx)
# print(tX)

# k_fold = 10
# k_indices = build_k_indices(y, k_fold)
    
# k = 2

# y_test = y[k_indices[k]]
# x_test = std_tx[k_indices[k]]
# train_indices = k_indices[~(np.arange(k_indices.shape[0]) == k)]
# y_train = np.concatenate(y[train_indices])
# x_train = np.concatenate(std_tx[train_indices])
    


In [83]:
max_iter = 35000
lambdas = np.arange(0.1, 0.4, 0.1)
gammas = np.array([0.005])

degree = 2

i_0, i_1, i_23 = split_dataset_wrt22(tX)

tx_0 =  tX[i_0]
y_0 =   y[i_0]
tx_1 =  tX[i_1]
y_1 =   y[i_1]
tx_23 = tX[i_23]
y_23 =  y[i_23]

std_tx_0 = standardize_0(tx_0)
std_tx_1 = standardize_1(tx_1)
std_tx_23 = standardize_23(tx_23)

matrix_std_tx_0 = build_poly(std_tx_0, degree)
matrix_std_tx_1 = build_poly(std_tx_1, degree)
matrix_std_tx_23 = build_poly(std_tx_23, degree)

In [84]:
weights_0 = reg_logistic_regression(y_0, matrix_std_tx_0, lambdas[0], gammas[0], max_iter)


Gamma:  0.00333333333333
Current iteration=0, the loss=69254.41425128581


/Users/chenfs/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: RuntimeWarning: overflow encountered in exp
/Users/chenfs/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:18: RuntimeWarning: overflow encountered in exp


Gamma:  8.67076495792e-05
Current iteration=100, the loss=907432.3983842771
Gamma:  3.85367331463e-05
Current iteration=200, the loss=579572.0798688746
Gamma:  3.85367331463e-05
Current iteration=300, the loss=444114.63556453993
Gamma:  1.71274369539e-05
Current iteration=400, the loss=342446.0555998906
Gamma:  1.71274369539e-05
Current iteration=500, the loss=299793.91725403134
Gamma:  1.71274369539e-05
Current iteration=600, the loss=260677.89710240118
Gamma:  1.71274369539e-05
Current iteration=700, the loss=223685.8381724672
Gamma:  1.71274369539e-05
Current iteration=800, the loss=188971.47363117422
Gamma:  1.71274369539e-05
Current iteration=900, the loss=157029.85037529917
Gamma:  1.71274369539e-05
Current iteration=1000, the loss=128488.11067916258
Gamma:  7.61219420174e-06
Current iteration=1100, the loss=111156.21268467171
Gamma:  7.61219420174e-06
Current iteration=1200, the loss=101549.91898673339
Gamma:  7.61219420174e-06
Current iteration=1300, the loss=93020.52449849749


In [87]:
weights_1 = reg_logistic_regression(y_1, matrix_std_tx_1, lambdas[0], gammas[0], max_iter)


Gamma:  0.00333333333333
Current iteration=0, the loss=53749.4049693404


/Users/chenfs/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: RuntimeWarning: overflow encountered in exp
/Users/chenfs/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:18: RuntimeWarning: overflow encountered in exp


Gamma:  0.000195092211553
Current iteration=100, the loss=1364469.6244029587
Gamma:  8.67076495792e-05
Current iteration=200, the loss=686902.4822888984
Gamma:  3.85367331463e-05
Current iteration=300, the loss=391479.3177926035
Gamma:  2.56911554309e-05
Current iteration=400, the loss=235798.0257714902
Gamma:  1.14182913026e-05
Current iteration=500, the loss=153538.7093832145
Gamma:  1.14182913026e-05
Current iteration=600, the loss=115915.75405120073
Gamma:  5.07479613449e-06
Current iteration=700, the loss=92898.236657372
Gamma:  5.07479613449e-06
Current iteration=800, the loss=82753.21929793632
Gamma:  5.07479613449e-06
Current iteration=900, the loss=74082.76167778182
Gamma:  5.07479613449e-06
Current iteration=1000, the loss=66702.97037265061
Gamma:  5.07479613449e-06
Current iteration=1100, the loss=60485.36097399944
Gamma:  5.07479613449e-06
Current iteration=1200, the loss=55370.46429210676
Gamma:  2.25546494866e-06
Current iteration=1300, the loss=52124.55261939003
Gamma:  

In [90]:
weights_23 = reg_logistic_regression(y_23, matrix_std_tx_23, lambdas[0], gammas[0], max_iter)


Gamma:  0.00333333333333
Current iteration=0, the loss=50282.97591936011


/Users/chenfs/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:18: RuntimeWarning: overflow encountered in exp
/Users/chenfs/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: RuntimeWarning: overflow encountered in exp


Gamma:  0.000130061474369
Current iteration=100, the loss=1299813.3253270006
Gamma:  5.78050997194e-05
Current iteration=200, the loss=821260.9207168942
Gamma:  5.78050997194e-05
Current iteration=300, the loss=616401.3876636017
Gamma:  5.78050997194e-05
Current iteration=400, the loss=420942.4316300268
Gamma:  3.85367331463e-05
Current iteration=500, the loss=286249.6861910225
Gamma:  1.71274369539e-05
Current iteration=600, the loss=206946.38208725894
Gamma:  1.71274369539e-05
Current iteration=700, the loss=157597.89235992503
Gamma:  1.71274369539e-05
Current iteration=800, the loss=113620.56760026883
Gamma:  7.61219420174e-06
Current iteration=900, the loss=85422.16557873185
Gamma:  7.61219420174e-06
Current iteration=1000, the loss=73176.41080177324
Gamma:  7.61219420174e-06
Current iteration=1100, the loss=63774.83872950742
Gamma:  7.61219420174e-06
Current iteration=1200, the loss=56729.12816050771
Gamma:  7.61219420174e-06
Current iteration=1300, the loss=51293.49741852834
Gamm

In [91]:
print("0  Size: ", len(y_0), "\tPerformance: ", performance(weights_0, y_0, matrix_std_tx_0))
print("1  Size: ", len(y_1), "\tPerformance: ", performance(weights_1, y_1, matrix_std_tx_1))
print("23 Size: ", len(y_23), "\tPerformance: ", performance(weights_23, y_23, matrix_std_tx_23))

0  Size:  99913 	Performance:  0.8369781710087776
1  Size:  77544 	Performance:  0.7879268544310327
23 Size:  72543 	Performance:  0.8077002605351309


## Generate predictions and save ouput in csv format for submission:

In [92]:
DATA_TEST_PATH = '../../data/test.csv' 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
i_0_test, i_1_test, i_23_test = split_dataset_wrt22(tX_test)

tx_0_test = tX_test[i_0_test]
tx_1_test = tX_test[i_1_test]
tx_23_test = tX_test[i_23_test]

std_tx_0_test = standardize_0(tx_0_test)
std_tx_1_test = standardize_1(tx_1_test)
std_tx_23_test = standardize_23(tx_23_test)

ids_0_test = ids_test[i_0_test]
ids_1_test = ids_test[i_1_test]
ids_23_test = ids_test[i_23_test]

output_path = '../../data/output0.csv'
y_pred_0 = predict_labels(weights_0, build_poly(std_tx_0_test, degree))
create_csv_submission(ids_0_test, y_pred_0, output_path)


output_path = '../../data/output1.csv'
y_pred_1 = predict_labels(weights_1, build_poly(std_tx_1_test, degree))
create_csv_submission(ids_1_test, y_pred_1, output_path)


output_path = '../../data/output23.csv'
y_pred_23 = predict_labels(weights_23, build_poly(std_tx_23_test, degree))
create_csv_submission(ids_23_test, y_pred_23, output_path)

In [167]:
OUTPUT_PATH = '../../data/output.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, build_poly(standardize(tX_test), degree))
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)